## BLEU - LLM

In [1]:
import pandas as pd

df = pd.read_csv("../agent/predictions.csv")
df = df.drop(columns=['debias_reasoning'])
df = df.rename(columns={"biases": "biases_detected","output":"output_agent"})
len(df)

853

In [2]:
df_causal = pd.read_csv("../../data/processed/20231220_metrics_CAUSAL.csv")
print('df_casual length: ',len(df_causal))
df = pd.merge(df, df_causal[['input','sesgo_pronombre','sesgo_otro','target','max_ref_len']], on='input', how='right')
print('Length resulting df after merging: ',len(df))

df_casual length:  782
Length resulting df after merging:  782


In [3]:
df['biases_detected'] = df['biases_detected'].fillna('UNBIASED')
df['output_agent'] = df['output_agent'].fillna('UNBIASED')

df['generation'] = df.apply(lambda row: row['input'] if row['biases_detected'] == 'UNBIASED' else row['output_agent'], axis=1)

df.head(3)

,input,biases_detected,scores,output_agent,index,sesgo_pronombre,sesgo_otro,target,max_ref_len,generation
0,Estimada comunidad beauchefiana: ¿Tienes papel...,UNBIASED,1.0,UNBIASED,0,NO,NO,['Estimada comunidad beauchefiana: ¿Tienes pap...,11,Estimada comunidad beauchefiana: ¿Tienes papel...
1,Desde hoy y hasta el 19 de diciembre puedes de...,GENERIC_PRONOUNS,0.8,Desde hoy y hasta el 19 de diciembre puedes de...,1,NaN,NaN,['Desde hoy y hasta el 19 de diciembre puedes ...,17,Desde hoy y hasta el 19 de diciembre puedes de...
2,Revisa en el afiche qué tipo de papeles puedes...,EXCLUSIONARY_TERMS,0.7,Revisa en el afiche qué tipo de papeles puedes...,2,NaN,NaN,['Revisa en el afiche qué tipo de papeles pued...,24,Revisa en el afiche qué tipo de papeles puedes...


* For metrics calc, targets with less than 5 tokens are discarded, to avoid 0 counts of 4-gram overlaps

In [4]:
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu

In [5]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /home/camilo/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [6]:
import numpy as np
import ast

df["reference_tokens"] = [
    [
        word_tokenize(tt)
        for tt in ast.literal_eval(t)
        if len(word_tokenize(tt)) >= 5
    ]
    for t in df["target"]
]

df["max_ref_len"] = [
    np.array([
        len(rti) for rti in rt
    ]).max() if len(rt) else 0
    for rt in df["reference_tokens"]
]

df_metric = df[df["max_ref_len"] >= 5].copy()
df_metric.shape, df.shape

((782, 11), (782, 11))

In [7]:
len(df[df["max_ref_len"] >= 5])

782

In [8]:
df_metric["generated_tokens"] = [
    word_tokenize(g)
    for g in df_metric["generation"]
]

df_metric["input_tokens"] = [
    word_tokenize(i)
    for i in df_metric["input"]
]

In [9]:
df_metric["bleu_gen"] = [
    sentence_bleu(t, g)
    for t, g in zip(df_metric["reference_tokens"], df_metric["generated_tokens"])
]

df_metric["bleu_input"] = [
    sentence_bleu(t, i)
    for t, i in zip(df_metric["reference_tokens"], df_metric["input_tokens"])
]

/home/camilo/miniconda3/envs/debiasing/lib/python3.11/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/camilo/miniconda3/envs/debiasing/lib/python3.11/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/camilo/miniconda3/envs/debiasing/lib/python3.11/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it cont

In [10]:
df_metric["bleu_gen"].describe().round(5), df_metric["bleu_input"].describe().round(5)

(count    782.00000
 mean       0.90391
 std        0.17307
 min        0.00000
 25%        0.84929
 50%        1.00000
 75%        1.00000
 max        1.00000
 Name: bleu_gen, dtype: float64,
 count    782.00000
 mean       0.99495
 std        0.02723
 min        0.59460
 25%        1.00000
 50%        1.00000
 75%        1.00000
 max        1.00000
 Name: bleu_input, dtype: float64)

In [11]:
print(round(corpus_bleu([t for t in df_metric["reference_tokens"]], df_metric["generated_tokens"]), 5))
print(round(corpus_bleu([t for t in df_metric["reference_tokens"]], df_metric["input_tokens"]), 5))

0.92029
0.99454


In [12]:
bajos = df_metric[df_metric["bleu_gen"] < 0.3]
print(bajos.shape)

for _, r in bajos.iterrows():
    print(f"Input: {r['input']}")
    print(f"Target: {r['target']}")
    print(f"Generation: {r['generation']}")
    print(f"BLEU: {round(r['bleu_gen'], 10)}")
    print()

(15, 15)
Input: Verifica y reúne la documentación que debes enviar de acuerdo a tu casoListado_de_documentos_para_acreditac.pdf (1.4 mb)Anexos.zip (2.8 mb)4.
Target: ['Verifica y reúne la documentación que debes enviar de acuerdo a tu casoListado_de_documentos_para_acreditac.pdf (1.4 mb)Anexos.zip (2.8 mb)4.']
Generation: Verifica y reúne la documentación que se debe enviar según el caso
BLEU: 0.1595194819

Input: CEIMEC   Centro de Estudiantes de Ingeniería Mecánica
Target: ['CEIMEC   Centro de Estudiantes de Ingeniería Mecánica']
Generation: CEIMEC   Centro de Estudiantado de Ingeniería Mecánica
BLEU: 0.0

Input: El número de consejero son 4.
Target: ['El número de consejero/a son 4.', 'El número de consejera/o son 4.', 'El número de consejero(a) son 4.', 'El número de consejera(o) son 4.', 'El número de consejer@ son 4.']
Generation: El número de consejeras y consejeros son 4.
BLEU: 0.0

Input: Escríbenos a bienestarestudiantil@ing.uchile.cl
Target: ['Escríbenos a bienestarestudiant

### Diffs

In [13]:
df_metric["bleu_dif"] = df_metric["bleu_gen"] - df_metric["bleu_input"]
df_metric["bleu_dif"].describe()

count    782.000000
mean      -0.091044
std        0.170253
min       -1.000000
25%       -0.145594
50%        0.000000
75%        0.000000
max        0.204729
Name: bleu_dif, dtype: float64

# Explore inputs with bias

In [14]:
df_with_bias = df_metric[
    (df_metric["sesgo_pronombre"] == "SI") | (df_metric["sesgo_otro"] == "SI") 
]

df_without_bias = df_metric[
    (df_metric["sesgo_pronombre"] == "NO") & (df_metric["sesgo_otro"] == "NO") 
]

df_not_able_to_bias = df_metric[
    (df_metric["sesgo_pronombre"].isna()) & (df_metric["sesgo_otro"].isna()) 
]

print(df_metric.shape, df_with_bias.shape[0] + df_without_bias.shape[0] + df_not_able_to_bias.shape[0])

print(
    f"Mean BLEU on biased inputs: {round(df_with_bias['bleu_gen'].mean(), 5)}\n"
    # f"Mean ROUGE on biased inputs: {round(df_with_bias['rouge'].mean(), 5)}\n\n"
    f"Mean BLEU on not biased inputs: {round(df_without_bias['bleu_gen'].mean(), 5)}\n"
    # f"Mean ROUGE on not biased inputs: {round(df_without_bias['rouge'].mean(), 5)}\n\n"
    f"Mean BLEU on inputs that can't be biased: {round(df_not_able_to_bias['bleu_gen'].mean(), 5)}\n"
    # f"Mean ROUGE on inputs that can't be biased: {round(df_not_able_to_bias['rouge'].mean(), 5)}\n"
)

(782, 16) 782
Mean BLEU on biased inputs: 0.84913
Mean BLEU on not biased inputs: 0.86309
Mean BLEU on inputs that can't be biased: 0.93534



In [15]:
for i, (_, r) in enumerate(df_with_bias[["input", "generation", "bleu_gen"]].iterrows()):
    print(i+1)
    print(f"Input: {r['input']}")
    print(f"Generation: {r['generation']}")
    print(f"BLEU: {r['bleu_gen']}")
    print()

1
Input: Estimados estudiantes de Pregrado,Junto con saludar les invitamos al OPEN MDS, charla Abierta para conocer los detalles del Magíster en Ciencia de Datos MDS de nuestra Facultad el cual es articulable con las carreras de pregrado FCFM.
Generation: Estimada comunidad estudiantil de Pregrado, junto con saludar les invitamos al OPEN MDS, charla abierta para conocer los detalles del Magíster en Ciencia de Datos MDS de nuestra Facultad el cual es articulable con las carreras de pregrado FCFM.
BLEU: 0.7984737721008085

2
Input: Los estudiantes que inscriban Prácticas Profesionales deberán pagar 2 créditos (arancel mínimo).La fecha de pago del arancel del Semestre de Verano será desde el 02/01/2024 al 05/01/2024, el detalle de cómo realizar el pago del arancel se informará durante los próximos días.
Generation: Quienes inscriban Prácticas Profesionales deberán pagar 2 créditos (arancel mínimo). La fecha de pago del arancel del Semestre de Verano será desde el 02/01/2024 al 05/01/2024,

## Metrics exploration

### No diff

In [16]:
wb_0 = df_with_bias[df_with_bias["bleu_dif"] == 0]
print(wb_0.shape)
wb_0["bleu_input"].describe()

(11, 16)


count    11.000000
mean      0.941323
std       0.063190
min       0.785629
25%       0.929007
50%       0.971815
75%       0.982239
max       0.993174
Name: bleu_input, dtype: float64

### Input more similar than generation

In [17]:
wb_below_0 = df_with_bias[df_with_bias["bleu_dif"] < 0]
wb_below_0.shape

(39, 16)

In [18]:
for i, r in wb_below_0.iterrows():
    print(i)
    print("I:", " ".join(r['input_tokens']))
    print("G:", " ".join(r['generated_tokens']))
    print(set(r['input_tokens']).symmetric_difference(set(r['generated_tokens'])))
    print()

11
I: Estimados estudiantes de Pregrado , Junto con saludar les invitamos al OPEN MDS , charla Abierta para conocer los detalles del Magíster en Ciencia de Datos MDS de nuestra Facultad el cual es articulable con las carreras de pregrado FCFM .
G: Estimada comunidad estudiantil de Pregrado , junto con saludar les invitamos al OPEN MDS , charla abierta para conocer los detalles del Magíster en Ciencia de Datos MDS de nuestra Facultad el cual es articulable con las carreras de pregrado FCFM .
{'Abierta', 'comunidad', 'Estimada', 'junto', 'Junto', 'Estimados', 'estudiantil', 'abierta', 'estudiantes'}

35
I: Los estudiantes que inscriban Prácticas Profesionales deberán pagar 2 créditos ( arancel mínimo ) .La fecha de pago del arancel del Semestre de Verano será desde el 02/01/2024 al 05/01/2024 , el detalle de cómo realizar el pago del arancel se informará durante los próximos días .
G: Quienes inscriban Prácticas Profesionales deberán pagar 2 créditos ( arancel mínimo ) . La fecha de pago

### Generation more similar than input

In [19]:
wb_above_0 = df_with_bias[df_with_bias["bleu_dif"] > 0]
print(wb_above_0.shape)
wb_above_0["bleu_gen"].describe()

(8, 16)


count    8.000000
mean     0.975394
std      0.069595
min      0.803155
25%      1.000000
50%      1.000000
75%      1.000000
max      1.000000
Name: bleu_gen, dtype: float64

In [20]:
not_1 = wb_above_0[wb_above_0["bleu_gen"] < 1]

for i, r in not_1.iterrows():
    print(i)
    print("I:", " ".join(r['input_tokens']))
    print("G:", " ".join(r['generated_tokens']))
    print(set(r['input_tokens']).symmetric_difference(set(r['generated_tokens'])))
    print()

501
I: 🗣️Oradores : Omar Bravo - Consultor Senior / Rosario Veas - Consultor Senior / Agustín Canete - Consultor Analista .
G: Oradores : Omar Bravo - Consultor Senior / Rosario Veas - Consultora Senior / Agustín Canete - Consultor Analista .
{'Consultora', '🗣️Oradores', 'Oradores'}



## Doesn't have bias

In [21]:
nb_0 = df_without_bias[df_without_bias["bleu_dif"] == 0]
print(nb_0.shape)
nb_0["bleu_input"].describe()

(77, 16)


count    77.0
mean      1.0
std       0.0
min       1.0
25%       1.0
50%       1.0
75%       1.0
max       1.0
Name: bleu_input, dtype: float64

In [22]:
not_1 = nb_0[nb_0["bleu_input"] < 1]

for i, r in not_1.iterrows():
    print(i)
    print("I:", " ".join(r['input_tokens']))
    print("G:", " ".join(r['generated_tokens']))
    print(set(r['input_tokens']).symmetric_difference(set(r['generated_tokens'])))
    print()

### Input more similar than generation

In [23]:
nb_below_0 = df_without_bias[df_without_bias["bleu_dif"] < 0]
nb_below_0.shape

(194, 16)

In [24]:
for i, r in nb_below_0.iterrows():
    print(i)
    print("I:", " ".join(r['input_tokens']))
    print("G:", " ".join(r['generated_tokens']))
    print(set(r['input_tokens']).symmetric_difference(set(r['generated_tokens'])))
    print()

7
I: Estimados/as estudiantes , Les recordamos a los/as estudiantes del plan de estudios v3 que cuentan con el beneficio BAES , y que realizarán su práctica profesional este verano , que pueden acercarse hasta mañana miércoles 6 de diciembre a la Secretaría Docente de sus departamentos para confirmar si fueron incluidos en la nómina que será enviada a Junaeb .
G: Estimada comunidad estudiantil , Les recordamos a los estudiantes del plan de estudios v3 que cuentan con el beneficio BAES , y que realizarán su práctica profesional este verano , que pueden acercarse hasta mañana miércoles 6 de diciembre a la Secretaría Docente de sus departamentos para confirmar si fueron incluidos en la nómina que será enviada a Junaeb .
{'comunidad', 'Estimada', 'los/as', 'Estimados/as', 'los', 'estudiantil'}

12
I: Les informaremos del cuerpo académico , el plan de estudio , las vías de admisión y requisitos necesarios para su ingreso .
G: Se informará sobre el cuerpo académico , el plan de estudio , las

### Generation more similar than input

In [25]:
nb_above_0 = df_without_bias[df_without_bias["bleu_dif"] > 0]
print(nb_above_0.shape)
nb_above_0["bleu_gen"].describe()

(0, 16)


count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: bleu_gen, dtype: float64

## Unable to bias

### No diff

In [26]:
ub_0 = df_not_able_to_bias[df_not_able_to_bias["bleu_dif"] == 0]
print(ub_0.shape)
ub_0["bleu_input"].describe()

(365, 16)


count    365.000000
mean       0.999769
std        0.003122
min        0.957064
25%        1.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: bleu_input, dtype: float64

In [27]:
not_1 = ub_0[ub_0["bleu_input"] < 1]

for i, r in not_1.iterrows():
    print(i)
    print("I:", " ".join(r['input_tokens']))
    print("G:", " ".join(r['generated_tokens']))
    print(set(r['input_tokens']).symmetric_difference(set(r['generated_tokens'])))
    print()

581
I: `` 2073 '' es la nueva obra musical de la compañía Hermanos Ibarra Roa y narra la historia de una mujer en tres versiones de sí misma : Ella ( del presente ) , la niña ( del pasado ) y la anciana ( del futuro ) , que a través de una grieta temporal , formada por un gran terremoto , viajarán en el tiempo dando origen a una retrospectiva desde 1973 , pasando por nuestro presente año 2023 , y proyectándose a un futuro Chile idealizado en el año 2073 , donde habitaremos un país que condenará con todo el poder del Estado los crímenes de lesa humanidad y será una potencia democrática a nivel mundial .
G: `` 2073 '' es la nueva obra musical de la compañía Hermanos Ibarra Roa y narra la historia de una mujer en tres versiones de sí misma : Ella ( del presente ) , la niña ( del pasado ) y la anciana ( del futuro ) , que a través de una grieta temporal , formada por un gran terremoto , viajarán en el tiempo dando origen a una retrospectiva desde 1973 , pasando por nuestro presente año 2

### Input more similar than generation

In [28]:
ub_below_0 = df_not_able_to_bias[df_not_able_to_bias["bleu_dif"] < 0]
ub_below_0.shape

(88, 16)

In [29]:
for i, r in ub_below_0.iterrows():
    print(i)
    print("I:", " ".join(r['input_tokens']))
    print("G:", " ".join(r['generated_tokens']))
    print(set(r['input_tokens']).symmetric_difference(set(r['generated_tokens'])))
    print()

1
I: Desde hoy y hasta el 19 de diciembre puedes deshacerte de ellos y reciclarlos con ReBeauchef .
G: Desde hoy y hasta el 19 de diciembre puedes deshacerte de estos materiales y reciclarlos con ReBeauchef .
{'ellos', 'estos', 'materiales'}

2
I: Revisa en el afiche qué tipo de papeles puedes traer , los estarán recibiendo en el edificio Escuela , bajo la escalera principal .
G: Revisa en el afiche qué tipo de papeles puedes traer , se estarán recibiendo en el edificio Escuela , bajo la escalera principal .
{'los', 'se'}

16
I: Este trabajo final debe cumplir los requisitos tanto del MDS como de la carrera .
G: Este trabajo final debe cumplir los requisitos tanto del MDS como de la carrera del programa .
{'programa'}

42
I: Por lo tanto , les dejamos un documento adjunto con una serie de preguntas/respuestas para tratar de aclarar las dudas que existen .
G: Por lo tanto , adjuntamos un documento con una serie de preguntas/respuestas para tratar de aclarar las dudas que existen .
{'adj

### Generation more similar than input

In [30]:
ub_above_0 = df_not_able_to_bias[df_not_able_to_bias["bleu_dif"] > 0]
print(ub_above_0.shape)
ub_above_0["bleu_gen"].describe()

(0, 16)


count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: bleu_gen, dtype: float64